# 모델 저장과 복원

## Import

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

## Load Mnist dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels
test_labels = test_labels

train_images = train_images.reshape(-1, 28 * 28) / 255.0
test_images = test_images.reshape(-1, 28 * 28) / 255.0
train_images.shape, test_images.shape

((60000, 784), (10000, 784))

## Define model

In [5]:
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(28*28,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(
        optimizer="adam",
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return model

In [6]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Train model and Save

In [8]:
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [9]:
model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59776/60000 [============================>.] - ETA: 0s - loss: 0.2203 - accuracy: 0.9345
Epoch 00001: saving model to training/cp.ckpt
60000/60000 [==============================] - 7s 113us/sample - loss: 0.2200 - accuracy: 0.9345 - val_loss: 0.1062 - val_accuracy: 0.9682
Epoch 2/10
59840/60000 [============================>.] - ETA: 0s - loss: 0.0981 - accuracy: 0.9694
Epoch 00002: saving model to training/cp.ckpt
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0984 - accuracy: 0.9694 - val_loss: 0.0792 - val_accuracy: 0.9749
Epoch 3/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.0687 - accuracy: 0.9788
Epoch 00003: saving model to training/cp.ckpt
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0687 - accuracy: 0.9788 - val_loss: 0.0702 - val_accuracy: 0.9788
Epoch 4/10
59840/60000 [============================>.] - ETA: 0s - loss: 0.0543 - accuracy: 0.

In [11]:
ls {checkpoint_dir}

checkpoint                   cp.ckpt.index
cp.ckpt.data-00000-of-00001


## Load trained model
훈련하지 않은 새로운 모델을 만들어 보겠습니다. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 합니다. 여기서는 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있습니다.

In [12]:
model = create_model()
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

10000/1 - 0s - loss: 2.4040 - accuracy: 0.0772
훈련되지 않은 모델의 정확도:  7.72%


In [13]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

10000/1 - 0s - loss: 0.0369 - accuracy: 0.9823
복원된 모델의 정확도: 98.23%


## Re-train model

In [14]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dit = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

In [15]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [16]:
ls {checkpoint_dir}

checkpoint                   cp.ckpt.index
cp.ckpt.data-00000-of-00001


## Load model and evaluate

In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training/cp.ckpt'

In [18]:
model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

10000/1 - 0s - loss: 0.0369 - accuracy: 0.9823
복원된 모델의 정확도: 98.23%


## 수동으로 가중치 저장하기

In [19]:
model.save_weights("./checkpoints/checkpoints")

In [20]:
model = create_model()
model.load_weights("./checkpoints/checkpoints")
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

10000/1 - 0s - loss: 0.0369 - accuracy: 0.9823
복원된 모델의 정확도: 98.23%


## 전체 모델 저장하기
model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다. 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있습니다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.
두 개의 포맷(SavedModel과 HDF5)으로 모델을 저장할 수 있습니다. 텐서플로의 SavedModel 포맷은 TF2.x에서기본 파일 포맷입니다. 하지만 HDF5 포맷으로 저장할 수도 있습니다.

### SavedModel 포맷
SavedModel 포맷은 모델을 직렬화하는 다른 방법입니다. 이 포맷으로 저장한 모델은 tf.keras.models.load_model로 복원할 수 있고 텐서플로 서빙과 호환됩니다.

In [25]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 104us/sample - loss: 0.2197 - accuracy: 0.9342
Epoch 2/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0979 - accuracy: 0.9700
Epoch 3/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0688 - accuracy: 0.9786
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0524 - accuracy: 0.9832
Epoch 5/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.0454 - accuracy: 0.9854


In [26]:
!mkdir -p saved_model
model.save("saved_model/my_model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [27]:
!ls saved_model

my_model


In [28]:
!ls saved_model/my_model

assets         saved_model.pb variables


In [30]:
new_model = tf.keras.models.load_model("saved_model/my_model")
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

10000/1 - 0s - loss: 0.0340 - accuracy: 0.9810
복원된 모델의 정확도: 98.10%
(10000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [32]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2215 - accuracy: 0.9352
Epoch 2/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0977 - accuracy: 0.9707
Epoch 3/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.0678 - accuracy: 0.9787
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0539 - accuracy: 0.9831
Epoch 5/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0428 - accuracy: 0.9866


In [33]:
model.save("my_model.h5")

In [34]:
new_model = tf.keras.models.load_model("my_model.h5")
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [35]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

10000/1 - 0s - loss: 0.0308 - accuracy: 0.9820
복원된 모델의 정확도: 98.20%
